In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.26.0.3  279.24 KiB  16      100.0%            c729ec26-6c63-4636-bca3-88aaaf624a14  rack1
UN  172.26.0.2  181.58 KiB  16      100.0%            69948696-e749-45c7-83f3-7bc61ae171ca  rack1
UN  172.26.0.4  286 KiB     16      100.0%            d90054af-c316-4c12-94f3-9cbe644194c4  rack1



In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [3]:
cass.execute("drop keyspace if exists weather")
cass.execute("""
create keyspace weather with
replication = {'class': 'SimpleStrategy', 'replication_factor': 3};
""")
cass.execute("use weather")

In [4]:
cass.execute("""
CREATE TYPE station_record (tmin int, tmax int)
""")

In [5]:
cass.execute("""
create table stations(
    id TEXT,
    name TEXT static,
    date DATE,
    record weather.station_record,
    PRIMARY KEY(id, date)
)
""")

In [6]:
#q1
cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [7]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e82784cb-e8e4-4e8f-b331-b36ce595a303;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [8]:
stations_df = spark.read.text("ghcnd-stations.txt")

#wisconsin_stations_df = stations_df.filter(stations_df.value.substr(0,2) == "US")
wisconsin_stations = stations_df.collect()

for row in wisconsin_stations:
    state = row.value[38:40].strip()
    if(state == 'WI'):
        id = row.value[0:11].strip()
        name = row.value[41:72].strip()
        name = name.replace("'", "''")
        query = f"INSERT INTO weather.stations (id, name) VALUES ('{id}', '{name}')"
        cass.execute(query)

cass.execute("SELECT COUNT(*) FROM weather.stations").one()

    

Row(count=1313)

In [9]:
#q2
result = cass.execute("SELECT name FROM weather.stations WHERE id = 'US1WIMR0003'").one()
result.name

'AMBERG 1.3 SW'

In [10]:
#q3
query = f"SELECT token(id) FROM weather.stations WHERE id = '{'USC00470273'}'"
result = cass.execute(query)
token = result.one()[0]
token

-9014250178872933741

In [11]:
#q4
import subprocess
ring = subprocess.check_output(["nodetool", "ring"]).decode("utf-8")
for line in ring.splitlines()[5:]:
    node_id = line[75:]
    if node_id and int(node_id) > token:
        ret = int(node_id)
        break
if not ret:
    ret = int(ring.splitlines()[5][75:])
ret

-8530598149243383117

In [12]:
!unzip -n records.zip

Archive:  records.zip


In [13]:
weather_df = spark.read.parquet("records.parquet")
weather_df.show(50)

+-----------+--------+-------+------+
|    station|    date|element| value|
+-----------+--------+-------+------+
|USW00014898|20220101|   TMAX| -32.0|
|USW00014898|20220102|   TMAX| -77.0|
|USW00014898|20220103|   TMAX| -60.0|
|USW00014898|20220104|   TMAX|   0.0|
|USW00014898|20220105|   TMAX| -16.0|
|USW00014898|20220106|   TMAX| -71.0|
|USW00014898|20220107|   TMAX| -71.0|
|USW00014898|20220108|   TMAX| -32.0|
|USW00014898|20220109|   TMAX| -27.0|
|USW00014898|20220110|   TMAX|-149.0|
|USW00014898|20220111|   TMAX| -16.0|
|USW00014898|20220112|   TMAX|   6.0|
|USW00014898|20220113|   TMAX|  11.0|
|USW00014898|20220114|   TMAX| -77.0|
|USW00014898|20220115|   TMAX| -99.0|
|USW00014898|20220116|   TMAX| -60.0|
|USW00014898|20220117|   TMAX| -21.0|
|USW00014898|20220118|   TMAX|  28.0|
|USW00014898|20220119|   TMAX|  28.0|
|USW00014898|20220120|   TMAX|-121.0|
|USW00014898|20220121|   TMAX| -77.0|
|USW00014898|20220122|   TMAX| -43.0|
|USW00014898|20220123|   TMAX|-110.0|
|USW00014898

In [14]:
from pyspark.sql.functions import col

weather_df = weather_df.filter((col("element") == "TMIN") | (col("element") == "TMAX")) \
    .select("station", "date", "element", "value") \
    .groupBy("station", "date") \
    .pivot("element") \
    .agg({"value": "first"}) \
    .withColumnRenamed("TMIN", "tmin") \
    .withColumnRenamed("TMAX", "tmax")

weather_df = weather_df.withColumn("date", col("date").cast("date"))

for row in weather_df.collect():
    station_id = row["station"]
    date = row["date"]
    tmin = row["tmin"]
    tmax = row["tmax"]

In [15]:
#q5
"""
from station_pb2_grpc import StationStub
import grpc
import station_pb2

server_address = 'localhost:5440'  # Update with your server address
channel = grpc.insecure_channel(server_address)
stub = StationStub(channel)

station_id = 'USW00014837'
max_temp_response = stub.StationMax(station_pb2.StationMaxRequest(station=station_id))

max_temp_response
"""

"\nfrom station_pb2_grpc import StationStub\nimport grpc\nimport station_pb2\n\nserver_address = 'localhost:5440'  # Update with your server address\nchannel = grpc.insecure_channel(server_address)\nstub = StationStub(channel)\n\nstation_id = 'USW00014837'\nmax_temp_response = stub.StationMax(station_pb2.StationMaxRequest(station=station_id))\n\nmax_temp_response\n"

In [16]:
spark = SparkSession.builder \
    .appName("Cassandra Spark Integration") \
    .config("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3") \
    .getOrCreate()

# Load data from Cassandra table into a DataFrame
stations_df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .option("keyspace", "weather") \
    .option("table", "stations") \
    .load()

stations_df.createOrReplaceTempView("stations")

24/04/19 03:53:11 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [17]:
#q6
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [18]:
avg_temp_diff_df = stations_df \
    .filter(col("record").isNotNull()) \
    .withColumn("temp_diff", col("record.tmax") - col("record.tmin")) \
    .groupBy("id") \
    .agg({"temp_diff": "avg"}) \
    .withColumnRenamed("avg(temp_diff)", "avg_temp_diff")

avg_temp_diff_dict = avg_temp_diff_df.rdd.map(lambda row: (row['id'], row['avg_temp_diff'])).collectAsMap()

In [19]:
#q7
avg_temp_diff_dict

{}

In [20]:
#q8
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.26.0.3  226.44 KiB  16      100.0%            c729ec26-6c63-4636-bca3-88aaaf624a14  rack1
UN  172.26.0.2  276.28 KiB  16      100.0%            69948696-e749-45c7-83f3-7bc61ae171ca  rack1
UN  172.26.0.4  232.82 KiB  16      100.0%            d90054af-c316-4c12-94f3-9cbe644194c4  rack1



In [21]:
#q9
"""
server_address = 'localhost:5440'  # Update with your server address
channel = grpc.insecure_channel(server_address)
stub = StationStub(channel)

station_id = 'USW00014837'
max_temp_response = stub.StationMax(station_pb2.StationMaxRequest(station=station_id))

max_temp_response
"""

"\nserver_address = 'localhost:5440'  # Update with your server address\nchannel = grpc.insecure_channel(server_address)\nstub = StationStub(channel)\n\nstation_id = 'USW00014837'\nmax_temp_response = stub.StationMax(station_pb2.StationMaxRequest(station=station_id))\n\nmax_temp_response\n"

In [22]:
#q10
"""
station_id = 'USW00014839'
date = '2024-04-18'
tmin = 10
tmax = 25

response = stub.RecordTemps(station_pb2.RecordTempsRequest(
    station=station_id,
    date=date,
    tmin=tmin,
    tmax=tmax
        ))

response.error
"""

"\nstation_id = 'USW00014839'\ndate = '2024-04-18'\ntmin = 10\ntmax = 25\n\nresponse = stub.RecordTemps(station_pb2.RecordTempsRequest(\n    station=station_id,\n    date=date,\n    tmin=tmin,\n    tmax=tmax\n        ))\n\nresponse.error\n"